In [1]:
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_perf_ranks
from yf_utils import _5_lookback_slices

verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_df_c = 'df_close_clean' 

In [2]:
df_c = pickle_load(path_data_dump, filename_pickled_df_c)
df_train, df_val, df_test = _2_split_train_val_test(df_c)
len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)

In [23]:
# _n_samples = 2  # number of tuples to create for iloc start_train:end_train:end_eval
_n_samples = 3  # number of tuples to create for iloc start_train:end_train:end_eval
_days_lookbacks = [120, 60, 30, 15]
_days_eval = 10

In [22]:
# create n_samples of iloc start_train:end_train:end_eval using max value in days_loobacks
max_lookback_slices = _3_random_slices(len_df_train, n_samples=_n_samples, days_lookback=max(_days_lookbacks), days_eval=_days_eval)
sets_lookback_slices = _5_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=_days_lookbacks, verbose=False)
sets_lookback_slices 

[[(788, 803, 813), (773, 803, 813), (743, 803, 813), (683, 803, 813)],
 [(576, 591, 601), (561, 591, 601), (531, 591, 601), (471, 591, 601)],
 [(285, 300, 310), (270, 300, 310), (240, 300, 310), (180, 300, 310)]]

In [24]:
# grp_perf_ranks = {}
grp_most_common_syms = []
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for lb_slices in sets_lookback_slices:  
  for lb_slice in lb_slices:  # lb_slice, e.g. (246, 276, 286)
    start_train = lb_slice[0]
    end_train = lb_slice[1]
    start_eval = end_train
    end_eval = lb_slice[2]
    lookback = end_train - start_train
    eval = end_eval - start_eval
    print(f'lb_slices:     {lb_slices}')
    print(f'lb_slice:      {lb_slice}')
    print(f'days lookback: {lookback}')
    print(f'days eval:     {eval}')    
    print(f'start_train:   {start_train}')
    print(f'end_train:     {end_train}')
    # print(f'start_eval:    {start_eval}')
    # print(f'end_eval:      {end_eval}')

    _df = df_train.iloc[start_train:end_train]
    perf_ranks, most_common_syms = _4_perf_ranks(_df, n_top_syms=10)
    # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
    print(f'perf_ranks: {perf_ranks}')  
    # most common symbols of perf_ranks 
    print(f'most_common_syms: {most_common_syms}')     
    # grp_perf_ranks[lookback] = perf_ranks
    print(f'+++ finish lookback slice {lookback} +++\n')
    grp_most_common_syms.append(most_common_syms)
    
  print(f'grp_most_common_syms: {grp_most_common_syms}')
  print(f'===== finish lookback slices {lb_slices} =====\n')

lb_slices:     [(788, 803, 813), (773, 803, 813), (743, 803, 813), (683, 803, 813)]
lb_slice:      (788, 803, 813)
days lookback: 15
days eval:     10
start_train:   788
end_train:     803
perf_ranks: {'period-15': {'r_CAGR/UI': array(['UNFI', 'TNK', 'TNP', 'FRO', 'INGN', 'EURN', 'NTGR', 'VSTO', 'TWI',
       'TDOC'], dtype=object), 'r_CAGR/retnStd': array(['UNFI', 'TNK', 'TNP', 'FRO', 'INGN', 'EURN', 'NTGR', 'VSTO',
       'TDOC', 'TWI'], dtype=object), 'r_retnStd/UI': array(['ATNI', 'UNFI', 'WERN', 'ORCL', 'SHEN', 'WMK', 'CGNX', 'MODV',
       'TNK', 'UPS'], dtype=object)}}
most_common_syms: [('UNFI', 3), ('TNK', 3), ('TNP', 2), ('FRO', 2), ('INGN', 2), ('EURN', 2), ('NTGR', 2), ('VSTO', 2), ('TWI', 2), ('TDOC', 2), ('ATNI', 1), ('WERN', 1), ('ORCL', 1), ('SHEN', 1), ('WMK', 1), ('CGNX', 1), ('MODV', 1), ('UPS', 1)]
+++ finish lookback slice 15 +++

lb_slices:     [(788, 803, 813), (773, 803, 813), (743, 803, 813), (683, 803, 813)]
lb_slice:      (773, 803, 813)
days lookback: 30
day

In [6]:
L = [('grape', 100), ('apple', 15), ('grape', 3), ('apple', 10),
    ('apple', 4), ('banana', 3)]

In [7]:
def grp_tuples_sort_sum(L, reverse=True):
  # https://stackoverflow.com/questions/2249036/grouping-python-tuple-list
  # https://stackoverflow.com/questions/10695139/sort-a-list-of-tuples-by-2nd-item-integer-value
  """
  Given a list of tuples of (key:value) such as:
  [('grape', 100), ('apple', 15), ('grape', 3), ('apple', 10),
   ('apple', 4), ('banana', 3)]
  Returns list of grouped-sorted-tuples based on summed-values such as:
  [('grape', 103), ('apple', 29), ('banana', 3)] 

  Args:
      L(list of tuples): list of tuples of key(str):value(int) pairs
      reverse(bool): sort order of summed-values of the grouped tuples,
       default is in descending order.  

  Return:
      grp_sorted_list(list of tuples): list of grouped-sorted-tuples
       based on summed-values such as:
       [('grape', 103), ('apple', 29), ('banana', 3)] 
  """ 

  import itertools
  from operator import itemgetter

  grp_list = []
  l = sorted(L)
  it = itertools.groupby(l, itemgetter(0))

  for key, subiter in it:
    # print(f'key: {key}')
    key_sum = sum(item[1] for item in subiter)
    # print(f'key_sum: {key_sum}')
    grp_list.append((key, key_sum))

  grp_sorted_list = sorted(grp_list, key=itemgetter(1), reverse=reverse)

  return grp_sorted_list  


In [8]:
grp_tuples_sort_sum(L, reverse=True)

[('grape', 103), ('apple', 29), ('banana', 3)]

In [10]:
# grp_perf_ranks
grp_most_common_syms
# flatten list of lists of (symbols:count)
flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]

sorted_grp_most_common_syms = grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
print(f'len(sorted_grp_most_common_syms): {len(sorted_grp_most_common_syms)}')
sorted_grp_most_common_syms
# top_common_syms = sorted_grp_most_common_syms[0:5]
# top_common_syms = [i[0] for i in top_common_syms]  # get keys from key:value pairs
# top_common_syms 

len(sorted_grp_most_common_syms): 95


[('CHRD', 8),
 ('AXSM', 6),
 ('ITCI', 6),
 ('FTSM', 5),
 ('ISEE', 5),
 ('TSLA', 5),
 ('BEEM', 4),
 ('CPRI', 4),
 ('VERU', 4),
 ('VICR', 4),
 ('APA', 3),
 ('CLDX', 3),
 ('JKS', 3),
 ('MARA', 3),
 ('MGI', 3),
 ('SGRY', 3),
 ('TPR', 3),
 ('APPS', 2),
 ('BGFV', 2),
 ('BMY', 2),
 ('CASH', 2),
 ('CNK', 2),
 ('CPE', 2),
 ('CVEO', 2),
 ('DAC', 2),
 ('DAR', 2),
 ('DMRC', 2),
 ('DQ', 2),
 ('ETN', 2),
 ('FATE', 2),
 ('FTNT', 2),
 ('HOV', 2),
 ('JHX', 2),
 ('LOB', 2),
 ('MRTX', 2),
 ('MYOV', 2),
 ('NBR', 2),
 ('PERI', 2),
 ('PLUG', 2),
 ('RENN', 2),
 ('SAGE', 2),
 ('SEED', 2),
 ('SM', 2),
 ('SPR', 2),
 ('SPWR', 2),
 ('SYNA', 2),
 ('TA', 2),
 ('AAPL', 1),
 ('ADEA', 1),
 ('ALGN', 1),
 ('APAM', 1),
 ('BBW', 1),
 ('BTI', 1),
 ('CACI', 1),
 ('COLL', 1),
 ('CRVL', 1),
 ('DBRG', 1),
 ('DEA', 1),
 ('DVA', 1),
 ('EDU', 1),
 ('EFR', 1),
 ('EVH', 1),
 ('FBNC', 1),
 ('FCX', 1),
 ('FFNW', 1),
 ('FLR', 1),
 ('FLXS', 1),
 ('GRMN', 1),
 ('HCKT', 1),
 ('IAC', 1),
 ('IMAX', 1),
 ('KB', 1),
 ('LDOS', 1),
 ('LE', 1),

In [11]:
top_common_syms = sorted_grp_most_common_syms[0:5]
top_common_syms = [i[0] for i in top_common_syms]  # get keys from key:value pairs
top_common_syms 

['CHRD', 'AXSM', 'ITCI', 'FTSM', 'ISEE']